# Seminar BA & QM
### Code Bol.com Case

Below you will find the code for the Bol.com case from group 8. 

Table of contents:

*  [Data preparation](#Data-preparation)
*  [Functions](#Functions)
*  [Models](#Models)

****

In [325]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Data Preparation
##### Load Data

In [302]:
#Import data
df_2019 = pd.read_csv('/Users/LV/Documents/ECONOMETRIE-ECO/MASTER/Seminar/2021_Case_Bol.com_Happy_Matches/data/data_2019.csv', low_memory = False)
df_2020 = pd.read_csv('/Users/LV/Documents/ECONOMETRIE-ECO/MASTER/Seminar/2021_Case_Bol.com_Happy_Matches/data/data_2020.csv', low_memory = False)

#Concat files
df_full = pd.concat([df_2019, df_2020])
df_full = df_full.reset_index(drop = True)

print('Total # records: ',df_full.shape[0])

In [343]:
#Rename columns
rename_columns = {'datetTimeFirstDeliveryMoment': 'dateTimeFirstDeliveryMoment',
                  'generalMatchClassification':   'detailedMatchClassification',
                  'detailedMatchClassification':  'generalMatchClassification',
                  'quanityReturned':              'quantityReturned'}

df_full = df_full.rename(columns = rename_columns)

In [304]:
#Transform dates to date-type
df_full['orderDate']                   = pd.to_datetime(df_full['orderDate'])
df_full['cancellationDate']            = pd.to_datetime(df_full['cancellationDate'])
df_full['promisedDeliveryDate']        = pd.to_datetime(df_full['promisedDeliveryDate'])
df_full['shipmentDate']                = pd.to_datetime(df_full['shipmentDate'])
df_full['dateTimeFirstDeliveryMoment'] = pd.to_datetime(df_full['dateTimeFirstDeliveryMoment'])
df_full['startDateCase']               = pd.to_datetime(df_full['startDateCase'])
df_full['returnDateTime']              = pd.to_datetime(df_full['returnDateTime'])
df_full['registrationDateSeller']      = pd.to_datetime(df_full['registrationDateSeller'])

##### Remove Noise

In [641]:
#Remove non-sensible rows
noise = df_full.loc[(df_full['startDateCase']        < df_full['orderDate']) | 
                   (df_full['cancellationDate']      < df_full['orderDate']) |
                   (df_full['promisedDeliveryDate']  < df_full['orderDate']) |
                   (df_full['shipmentDate']          < df_full['orderDate']) |
                   (df_full['dateTimeFirstDeliveryMoment'].dt.normalize() < df_full['orderDate']) |
                   (df_full['returnDateTime']        < df_full['orderDate']) |
                   (df_full['orderDate']             < df_full['registrationDateSeller']) |
                   (df_full['cancellationDate']      > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   (df_full['cancellationDate']      > df_full['returnDateTime']) |
                   (df_full['shipmentDate']          > df_full['returnDateTime']) |
                   (df_full['shipmentDate']          > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   (df_full['registrationDateSeller'].isnull()) |
                   (df_full['promisedDeliveryDate'].isnull()) |
                   ((df_full['returnDateTime']       < df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) & 
                            (df_full['dateTimeFirstDeliveryMoment'].notnull()) &
                            (df_full['returnDateTime'].notnull())) |
                   ((df_full['cancellationDate']     > df_full['shipmentDate']) &
                            ((df_full['cancellationReasonCode'] == 'CUST_FE') | (df_full['cancellationReasonCode'] == 'CUST_CS')))].index

#Drop noise data
df = df_full.drop(index = noise)
print(len(noise), 'complete records removed from the data')
print('Cleaned # records: ',df.shape[0],'\n')

#Sort rows on orderDate and create new index
df = df.sort_values(by = 'orderDate')
df = df.reset_index(drop = True)

#Remove inconsistent values
cancellationNoise = df.loc[(df['noCancellation'] == True) & (df['cancellationDate'].notnull())].index
returnNoise       = df.loc[(df['noReturn'] == True) & (df['returnDateTime'].notnull())].index
caseNoise         = df.loc[(df['noCase'] == True) & (df['startDateCase'].notnull())].index
quantityNoise     = df.loc[df['quantityReturned'] > df['quantityOrdered']].index
deliveryNoise     = df.loc[(df['dateTimeFirstDeliveryMoment'].notnull()) & (df['onTimeDelivery'].isnull())].index

df.loc[cancellationNoise, ['cancellationDate','cancellationReasonCode']] = None
df.loc[returnNoise,       ['returnDateTime','quantityReturned','returnCode']] = None
df.loc[caseNoise,         ['startDateCase','cntDistinctCaseIds','hasOneCase','hasMoreCases']] = None
df.loc[quantityNoise,     ['quantityReturned']] = df.loc[quantityNoise, ['quantityOrdered']]
df.loc[deliveryNoise,     ['dateTimeFirstDeliveryMoment']] = None

print('# Records where cancellation values are emptied:',len(cancellationNoise))
print('# Records where return values are emptied: \t',len(returnNoise))
print('# Records where case values are emptied: \t',len(caseNoise))
print('# Records where quantity values are equalized: \t',len(quantityNoise))
print('# Records where delivery values are emptied: \t',len(deliveryNoise))

6516 complete records removed from the data
Cleaned # records:  4772950 

# Records where cancellation values are emptied: 53780
# Records where return values are emptied: 	 8208
# Records where case values are emptied: 	 0
# Records where quantity values are equalized: 	 14722
# Records where delivery values are emptied: 	 4103


##### Create New Variables

In [578]:
#Create new variables related to periods (days difference)
df['caseDays']             = (df['startDateCase'] - df['orderDate']).dt.days
df['returnDays']           = (df['returnDateTime'] - df['orderDate']).dt.days
df['cancellationDays']     = (df['cancellationDate'] - df['orderDate']).dt.days
df['actualDeliveryDays']   = (df['dateTimeFirstDeliveryMoment'].dt.normalize() - df['orderDate']).dt.days
df['shipmentDays']         = (df['shipmentDate'] - df['orderDate']).dt.days
df['partnerSellingMonths'] = (df['orderDate'] - df['registrationDateSeller']).dt.days
df['promisedDeliveryDays'] = (df['promisedDeliveryDate'] - df['orderDate']).dt.days

#Time related variables
df['orderYear']    = df['orderDate'].dt.year
df['orderMonth']   = df['orderDate'].dt.month
df['orderWeekday'] = df['orderDate'].dt.weekday
df['orderCorona']  = df['orderDate'].apply(lambda x: True if x > datetime.strptime('2020-03-20','%Y-%m-%d') else False)

#Create dummy variables for weeday and months
df['orderMonday']    = df['orderWeekday'].apply(lambda x: True if x == 1 else False)
df['orderTuesday']   = df['orderWeekday'].apply(lambda x: True if x == 2 else False)
df['orderWednesday'] = df['orderWeekday'].apply(lambda x: True if x == 3 else False)
df['orderThursday']  = df['orderWeekday'].apply(lambda x: True if x == 4 else False)
df['orderFriday']    = df['orderWeekday'].apply(lambda x: True if x == 5 else False)
df['orderSaturday']  = df['orderWeekday'].apply(lambda x: True if x == 6 else False)
df['orderSunday']    = df['orderWeekday'].apply(lambda x: True if x == 7 else False)

df['orderJanuary']   = df['orderMonth'].apply(lambda x: True if x == 1 else False)
df['orderFebruary']  = df['orderMonth'].apply(lambda x: True if x == 2 else False)
df['orderMarch']     = df['orderMonth'].apply(lambda x: True if x == 3 else False)
df['orderApril']     = df['orderMonth'].apply(lambda x: True if x == 4 else False)
df['orderMay']       = df['orderMonth'].apply(lambda x: True if x == 5 else False)
df['orderJune']      = df['orderMonth'].apply(lambda x: True if x == 6 else False)
df['orderJuly']      = df['orderMonth'].apply(lambda x: True if x == 7 else False)
df['orderAugust']    = df['orderMonth'].apply(lambda x: True if x == 8 else False)
df['orderSeptember'] = df['orderMonth'].apply(lambda x: True if x == 9 else False)
df['orderOctober']   = df['orderMonth'].apply(lambda x: True if x == 10 else False)
df['orderNovember']  = df['orderMonth'].apply(lambda x: True if x == 11 else False)
df['orderDecember']  = df['orderMonth'].apply(lambda x: True if x == 12 else False)

#Other variables
df['productTitleLength'] = len(df['productTitle'])

df['countryCodeNL']   = df['countryCode'].apply(lambda x: True if x == 'NL' else False)
df['fulfilmentByBol'] = df['fulfilmentType'].apply(lambda x: True if x == 'FBB' else False)
df['countryOriginNL'] = df['countryOriginSeller'].apply(lambda x: True if x == 'NL' else False)
df['countryOriginBE'] = df['countryOriginSeller'].apply(lambda x: True if x == 'BE' else False)
df['countryOriginDE'] = df['countryOriginSeller'].apply(lambda x: True if x == 'DE' else False)

#determinantClassification

#transporterCluster

#productCluster

In [579]:
#Change type of columns
dtype = {'calculationDefinitive': bool,
         'noCancellation': bool,
         'noCase': bool,
         'hasOneCase': bool,
         'hasMoreCases': bool,
         'noReturn': bool}

df = df.astype(dtype)
df.dtypes

orderDate          datetime64[ns]
productId                   int64
sellerId                    int64
totalPrice                float64
quantityOrdered             int64
                        ...      
countryCodeNL                bool
fulfilmentByBol              bool
countryOriginNL              bool
countryOriginBE              bool
countryOriginDE              bool
Length: 74, dtype: object

### Functions

In [623]:
def addPeriodColumns(df,X):
    """
    Function to create columns which indicate whether determinants are known after X days.
    Input: X = number of days after order date at which the prediction is made
           df = dataFrame
    TO-DO: add transporter dummies if known after shipmentDate
    """
    
    df['caseKnown']           = df.apply(lambda row: True if row.caseDays <= X else False, axis = 1)
    df['returnKnown']         = df.apply(lambda row: True if row.returnDays <= X else False, axis = 1)
    df['cancellationKnown']   = df.apply(lambda row: True if row.cancellationDays <= X else False, axis = 1)
    df['onTimeDeliveryKnown'] = df.apply(lambda row: True if ((row.actualDeliveryDays <= X) and (row.onTimeDelivery == True)) else False, axis = 1)
    df['lateDeliveryKnown']   = df.apply(lambda row: True if ((row.actualDeliveryDays <= X) and (row.onTimeDelivery == False)) else False, axis = 1)
    
#     df['transporterPostNL'] = df.apply(lambda row: True if ((row.shipmentDays <= X) and (row. == 'PostNL')) else False, axis = 1)
#     df['transporterDHL']    = df.apply(lambda row: True if ((row.shipmentDays <= X) and (row. == 'DHL')) else False, axis = 1)
#     df['transporterGLS']    = df.apply(lambda row: True if ((row.shipmentDays <= X) and (row. == 'GLS')) else False, axis = 1)
#     df['transporterBrief']  = df.apply(lambda row: True if ((row.shipmentDays <= X) and (row. == 'Brief')) else False, axis = 1)
#     df['transporterOther']  = df.apply(lambda row: True if ((row.shipmentDays <= X) and (row. == 'Other')) else False, axis = 1)

    return df

In [572]:
def addProductColumns(df):
    """
    Function to add 4 columns: productOrderCount, productTotalCount, productTotalReturned and productReturnFraction.
    Input: dataFrame with columns: 'productId','orderDate','quantityOrdered','quantityReturned','returnDateTime'.
    """
    df = df.sort_values(by = ['productId','orderDate'])
    df = df.reset_index(drop = True)
    
    df_ = df[['productId','orderDate','quantityOrdered','quantityReturned','returnDateTime']]
    
    #ProductTotalCount
    pivot = df_.groupby(['productId','orderDate']).quantityOrdered.sum().groupby('productId').cumsum()
    productTotalCount = df_.merge(pivot, 
                                left_on=['productId','orderDate'], 
                                right_index=True, 
                                how = 'left').quantityOrdered_y
    
    #ProductOrderCount
    pivot = df_.groupby(['productId','orderDate']).quantityOrdered.count().groupby('productId').cumsum()
    productOrderCount = df_.merge(pivot, 
                                left_on=['productId','orderDate'], 
                                right_index=True, 
                                how = 'left').quantityOrdered_y
    
    #ProductTotalReturned
    productTotalReturned = np.zeros(df_.shape[0])
    
    previousID = None
    
    returnDic = {}
    
    for row in df_.itertuples(): #iterate through dataFrame
        
        if row[0] == 0: 
            
            #update return dictionary if this product is returned
            if row[4] != None:
                if row[5] in returnDic:
                    returnDic[row[5]] += row[4]
                else:
                    returnDic[row[5]] = row[4]

            previousID = row[1]
            
        elif (previousID == row[1]):
            
            #update return dictionary if this product is returned
            if row[4] != None:
                if row[5] in returnDic:
                    returnDic[row[5]] += row[4]
                else:
                    returnDic[row[5]] = row[4]
            
            #add returned products to new dictionary if known
            known = {k: v for k, v in returnDic.items() if k <= row[2]}
            productTotalReturned[row[0]] = sum(known.values())
            
            #update the dictionary by removing the returns which are now known
            returnDic = {k: v for k, v in returnDic.items() if k > row[2]}
                        
            previousID = row[1]
            
        else:
            returnDic = {} #new productId, hence empty the return dictionary
            
            #update return dictionary if this product is returned
            if row[4] != None:
                if row[5] in returnDic:
                    returnDic[row[5]] += row[4]
                else:
                    returnDic[row[5]] = row[4]
                    
            previousID = row[1]
    
    df_['productTotalReturned'] = productTotalReturned
    pivot = df_.groupby(by = ['productId','orderDate']).productTotalReturned.sum().groupby('productId').cumsum()
    productTotalReturned = df_.merge(pivot, 
                                left_on=['productId','orderDate'], 
                                right_index=True, 
                                how = 'left').productTotalReturned_y
     
    #Add new columns to dataFrame    
    df['productOrderCount'] = productOrderCount
    df['productTotalCount'] = productTotalCount
    df['productTotalReturned'] = productTotalReturned
    df['productReturnFraction'] = productTotalReturned / productTotalCount
    
    return(df)

In [461]:
# def addProductCount_OLD(df):
#     """
#     Fuction to add a column to your dataframe which counts the number of occurances of
#     the productId uptil that date and total quantityOrdered.
#     Now 26 minutes for 1,000,000 rows
#     """
#     df = df.sort_values(by = ['productId','orderDate'])
#     df = df.reset_index(drop = True)
    
#     df_ = df[['productId','orderDate','quantityOrdered','quantityReturned','returnDateTime']]
    
#     ID_list = df_.productId.value_counts().sort_index().cumsum()
#     #count_ID = 0
    
#     productOrderCount = np.ones(df_.shape[0])
#     productTotalCount = np.zeros(df_.shape[0])
#     productTotalReturned = np.zeros(df.shape[0])
    
#     for row in df_.itertuples():
        
#         if row[0] == 0:
#             #df_P = df_.loc[df_['productId'] == row[1]]
#             #df_P = df_.iloc[:ID_list.iloc[count_ID]]
#             df_P = df_.iloc[:ID_list.loc[row[1]]]
            
#             productTotalCount[row[0]] = row[3]
            
#             previousId = row[1]
            
#         elif previousId == row[1]:
#             extract = df_P.loc[df_P['orderDate'] <= row[2]]
#             extract2 = extract.loc[extract['returnDateTime'] <= row[2]]
            
#             productOrderCount[row[0]] = extract.quantityOrdered.count()
#             productTotalCount[row[0]] = extract.quantityOrdered.sum()
#             productTotalReturned[row[0]] = extract2.quantityReturned.sum()
            
#             previousId = row[1]
            
#         else:
#             #df_P = df_.loc[df_['productId'] == row[1]]
#             #df_P = df_.iloc[ID_list.iloc[count_ID]:ID_list.iloc[count_ID+1]]
#             df_P = df_.iloc[ID_list.loc[previousId]:ID_list.loc[row[1]]]
            
#             extract = df_P.loc[df_P['orderDate'] <= row[2]]
#             extract2 = extract.loc[extract['returnDateTime'] <= row[2]]
            
#             productOrderCount[row[0]] = extract.quantityOrdered.count()
#             productTotalCount[row[0]] = extract.quantityOrdered.sum()
#             productTotalReturned[row[0]] = extract2.quantityReturned.sum()
            
#             previousId = row[1]
#             #count_ID += 1
               
#     df['productOrderCount'] = productOrderCount
#     df['productTotalCount'] = productTotalCount
#     df['productTotalReturned'] = productTotalReturned
#     df['productReturnFraction'] = productTotalReturned / productTotalCount
    
#     return(df)

#### TEST AREA

In [ ]:
#df.loc[(df['productId'] == 9300000000762808) | (df['productId'] == 9200000101821794)]
#df.loc[(df['productId'] == 9200000105019922)]
#[['productId','orderDate','returnDateTime','quantityOrdered','quantityReturned']]
#[['caseDays','returnDays','cancellationDays','actualDeliveryDays']]

# t0 = time.time()
# a = addProductColumns(df)
# t1 = time.time()
# total = t1-t0
# print(total)

In [630]:
df_ = df.loc[(df['productId'] == 9300000000762808) | (df['productId'] == 9200000101821794)]
#df_ = df.loc[(df['productId'] == 9200000105019922)]
df_ = df_[['productId','orderDate','returnDateTime','quantityOrdered','quantityReturned',
           'caseDays','returnDays','cancellationDays','actualDeliveryDays','onTimeDelivery']]

df_ = addProductColumns(df_)
df_ = addPeriodColumns(df_,2)
df_

<ipython-input-572-58034f0a03de>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['productTotalReturned'] = productTotalReturned


,productId,orderDate,returnDateTime,quantityOrdered,quantityReturned,caseDays,returnDays,cancellationDays,actualDeliveryDays,onTimeDelivery,productOrderCount,productTotalCount,productTotalReturned,productReturnFraction,caseKnown,returnKnown,cancellationKnown,onTimeDeliveryKnown,lateDeliveryKnown
0,9200000101821794,2019-02-19,NaT,1,NaN,NaN,NaN,NaN,1.0,True,1,1,0.0,0.000000,False,False,False,True,False
1,9200000101821794,2019-02-20,NaT,1,NaN,NaN,NaN,NaN,1.0,True,2,2,0.0,0.000000,False,False,False,True,False
2,9200000101821794,2019-03-19,NaT,1,NaN,NaN,NaN,NaN,1.0,True,3,3,0.0,0.000000,False,False,False,True,False
3,9200000101821794,2019-04-02,NaT,1,NaN,NaN,NaN,NaN,1.0,True,4,4,0.0,0.000000,False,False,False,True,False
4,9200000101821794,2019-04-14,NaT,1,NaN,NaN,NaN,NaN,1.0,True,5,5,0.0,0.000000,False,False,False,True,False
5,9200000101821794,2019-05-05,NaT,1,NaN,NaN,NaN,NaN,1.0,True,6,6,0.0,0.000000,False,False,False,True,False
6,9200000101821794,2019-06-09,NaT,1,NaN,NaN,NaN,NaN,2.0,True,7,7,0.0,0.000000,False,False,False,True,False
7,9200000101821794,2019-06-10,NaT,1,2.0,NaN,NaN,NaN,2.0,False,8,8,0.0,0.000000,False,False,False,False,True
8,9200000101821794,2019-06-16,NaT,1,NaN,NaN,NaN,NaN,1.0,True,9,9,0.0,0.000000,False,False,False,True,False
9,9200000101821794,2019-06-17,NaT,1,NaN,NaN,NaN,NaN,1.0,True,10,10,0.0,0.000000,False,False,False,True,False


### Models